# Machine Learning in Spark

In [1]:
!pip install mlflow

     |████████████████████████████████| 14.2 MB 4.0 MB/s eta 0:00:01
  Using cached sqlparse-0.4.1-py3-none-any.whl (42 kB)
     |████████████████████████████████| 159 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 146 kB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 370 kB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 5.3 MB/s  eta 0:00:01
     |████████████████████████████████| 94 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 6.7 MB/s  eta 0:00:01
     |████████████████████████████████| 61 kB 9.4 MB/s  eta 0:00:01
     |████████████████████████████████| 75 kB 8.3 MB/s  eta 0:00:01
  Created wheel for gunicorn: filename=gunicorn-20.1.0-py3-none-any.whl size=78918 sha256=da1dfe37d4c6ecac53c28f0bd832aa54c83389036e6680f040ccf9e2770dc7ee
  Stored in directory: /Users/zachgreenberg/Library/Caches/pip/wheels/21/4b/32/9be8daf8a4d73da26e4dba66c47c9b4

In [2]:
import pyspark
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import feature
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

# un-comment the following lines if running locally
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Exception: Java gateway process exited before sending its port number

## Agenda

SWBAT:

- use `pyspark` to build machine learning models.

This example assumes that we have a holdout validation dataset somewhere else, so we don't need to perform a train-test split, we only need to perform cross validation

Follow [these instructions](https://docs.databricks.com/data/data.html#import-data-1) to import `US_births_2000-2014_SSA.csv` into Databricks

In [ ]:
# this file path will be different if you are running Spark locally
df = spark.read.format('csv').option('header', 'true').\
load('/FileStore/tables/US_births_2000_2014_SSA-daa0e.csv')

In [ ]:
df.toPandas().head(3)

In [ ]:
df.dtypes

In [ ]:
df = df.withColumn('births', df['births'].cast('int'))
df = df.withColumn('day_of_week', df['day_of_week'].cast('int'))
df = df.withColumn('date_of_month', df['date_of_month'].cast('int'))
df = df.withColumn('month', df['month'].cast('int'))
df = df.withColumn('year', df['year'].cast('int'))

In [ ]:
ohe = feature.OneHotEncoder(inputCols=['date_of_month',
                                                'day_of_week'],
                                     outputCols=['date_vec',
                                                  'day_vec'],
                                     dropLast=True)
one_hot_encoded = ohe.fit(df).transform(df)
one_hot_encoded.head()

Note the 'SparseVector' we've created!

In [ ]:
features = ['year', 'month', 'date_of_month', 'day_of_week']

target = 'births'

vector = VectorAssembler(inputCols=features, outputCol='features')
vectorized_df = vector.transform(one_hot_encoded)

The Vector Assembler is often what we want when we're building a model in Spark. [How does the VectorAssembler work?](https://spark.apache.org/docs/2.1.0/ml-features.html#vectorassembler)

In [ ]:
vectorized_df.columns

In [ ]:
rf_model = RandomForestRegressor(featuresCol='features',
                                 labelCol='births',
                                 predictionCol="prediction").fit(vectorized_df)

In [ ]:
predictions = rf_model.transform(vectorized_df).select("births", "prediction")
predictions.head(3)

Let's evaluate our model! [Here](https://spark.apache.org/docs/2.2.0/mllib-evaluation-metrics.html) is a reference for the many metrics available in Spark.

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='births')

evaluator.evaluate(predictions, {evaluator.metricName:"r2"})

In [ ]:
evaluator.evaluate(predictions, {evaluator.metricName:"mae"})

In [ ]:
one_hot_encoder = OneHotEncoder(inputCols=['date_of_month',
                                                'day_of_week'],
                                     outputCols=['date_vec',
                                                  'day_vec'],
                                     dropLast=True)
vector_assember = VectorAssembler(inputCols=features,
                                  outputCol='features')
random_forest = RandomForestRegressor(featuresCol='features',
                                      labelCol='births')
stages = [one_hot_encoder, vector_assember, random_forest]

pipeline = Pipeline(stages=stages)

Note: The stages in a pipeline can be either *Transformers* or *Estimators*. An estimator fits a DataFrame to produce a Transformer.

In [ ]:
random_forest.params

In [ ]:
params = ParamGridBuilder().addGrid(random_forest.numTrees, [20,50,100]).build()

In [ ]:
reg_evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='births',
                                    metricName = 'mae')

In [ ]:
cv = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=params,
    evaluator=reg_evaluator,
    parallelism=4
)

In [ ]:
cross_validated_model = cv.fit(df.cache())

In [ ]:
cross_validated_model.avgMetrics

In [ ]:
cross_validated_model.

## .bestModel

In [ ]:
cross_validated_model.bestModel.stages

In [ ]:
cross_validated_model.bestModel.stages[2].getNumTrees

## Challenge

Look at [this documentation](https://docs.databricks.com/data/databricks-datasets.html) to find large datasets that come pre-loaded on DBFS (Databricks file system).  Choose one, and build an ML model based on it.

In [ ]:
display(dbutils.fs.ls("/databricks-datasets"))

In [ ]:
display(dbutils.fs.ls("/databricks-datasets/Rdatasets/"))

In [ ]:
dbutils.fs.ls('dbfs:/databricks-datasets')

In [ ]:
new = spark.read.format('csv')

In [ ]:
newdf = new.load('dbfs:/databricks-datasets/COVID/IHME/2020_08_06/'\
'Best_mask_hospitalization_all_locs.csv')

In [ ]:
newdf.toPandas()